# <center>**Taller 05:** Herramientas para indexación</center>

### Integrantes:
- Calahorrano David
- Córdova Carlos
- Zambrano Andrés

### 

### **Objetivo:**
Este ejercicio te permitirá recordar los conceptos fundamentales de indexación clásica en sistemas de recuperación de información. Implementarás un índice invertido manualmente y luego explorarás herramientas como Whoosh y Elasticsearch para construir y consultar índices.

### **Requisitos:**
1. Librerías Python necesarias: Whoosh, ElasticSearch.
2. Instalar Elasticsearch en un contenedor utilizando Docker.
3. Dataset Wikipedia Movie Plots, disponible en Kaggle.

-----

### Conceptos Clave
#### Indexación Invertida
- **Definición:** Un índice inverso es una estructura de datos que mapea cada palabra a la lista de documentos en los que aparece, junto con sus ubicaciones. Es fundamental en los sistemas de recuperación de información porque permite realizar búsquedas eficientes.
- **Ejemplo:** Si la palabra "cyborg" aparece en los documentos 1 y 3, el índice se vería así:
("cyborg", [1, 3])
- **Uso:** Facilita encontrar todos los documentos relevantes para una consulta.

#### Normalización de Texto
- **Definición:** Proceso de transformar texto para hacerlo uniforme, lo que incluye convertirlo a minúsculas, eliminar puntuación y caracteres especiales, entre otros.
- **Propósito:** Evitar inconsistencias en el procesamiento del texto (e.g., "Cyborg" y "cyborg" deben tratarse como iguales).
- **Pasos comunes:**
1. Convertir a minúsculas.
2. Eliminar puntuación.
3. Tokenizar en palabras.

#### Tokenización
- **Definición:** Dividir un texto en unidades básicas, generalmente palabras o frases.
- **Ejemplo:** El texto "Jurassic Park es una película." se tokeniza como:
["Jurassic", "park", "es", "una", "película"]
- **Herramientas:** En Python, librerías como NLTK o SpaCy pueden facilitar este proceso.

#### Whoosh
- **Definición:** Biblioteca de Python para construir motores de búsqueda ligeros, ideal para datasets pequeños.
- **Usos:** Indexación y búsqueda local sin necesidad de configurar servidores externos.
- **Ventajas:** Rápido y sencillo para proyectos individuales o educativos.

#### Elasticsearch
- **Definición:** Un motor de búsqueda de texto completo basado en Apache Lucene, diseñado para manejar grandes volúmenes de datos.

- **Usos:**
  - Crear índices distribuidos.
  - Realizar consultas complejas con operadores booleanos y puntuación de relevancia.

- **Ventajas:** Escalabilidad y velocidad en la recuperación de información en grandes colecciones de datos.

#### Consultas Básicas
- **Definición:** Proceso de buscar información en un índice utilizando palabras clave.
- **Ejemplos comunes:**
  - Match query: Buscar documentos que contengan una palabra clave.
  - Boolean query: Combinar palabras clave con operadores como "AND" u "OR".
- **Ejemplo con Elasticsearch:** Una consulta para buscar "dinosaurs AND park":
  ```json
  {
    "query": {
      "bool": {
        "must": [
          { "match": { "content": "dinosaurs" }},
          { "match": { "content": "park" }}
        ]
      }
    }
  }
  ```

---

### Preparación del Dataset: Wikipedia Movie Plots
1. Descarga del dataset:
   - Ve a Kaggle y busca el dataset "Wikipedia Movie Plots". Descarga el dataset.
2. Comprender el dataset:
   - El dataset contiene información sobre tramas de más de 34,000 películas. Los campos principales incluyen:
     - Release Year: Año de lanzamiento.
     - Title: Título de la película.
     - Plot: Descripción de la trama de la película.
   - Asegúrate de tener un archivo estructurado con las columnas Title y Plot, ya que trabajarás principalmente con estas.
   
   Ejemplo:

   | Title           | Plot                                                                                         |
   |------------------|----------------------------------------------------------------------------------------------|
   | The Terminator  | A cyborg assassin is sent back in time to kill the mother of a future resistance leader.      |
   | Jurassic Park   | Scientists clone dinosaurs to create a theme park, but things go wrong.                      |


In [ ]:
import kagglehub

# Descargar la última versión del dataset
path = kagglehub.dataset_download("jrobischon/wikipedia-movie-plots")

# Mostrar la ubicación de los archivos descargados
print("Path to dataset files:", path)

In [1]:
import pandas as pd

# Ruta del archivo descargado
file_path = r'C:\Users\USER\.cache\kagglehub\datasets\jrobischon\wikipedia-movie-plots\versions\1\wiki_movie_plots_deduped.csv'

# Cargar el archivo CSV
df = pd.read_csv(file_path)

# Seleccionar las columnas necesarias
table = df[['Title', 'Plot']]

# Mostrar las primeras 10 filas en formato Markdown
from IPython.display import Markdown, display

def show_markdown_table(dataframe):
    """Convierte un DataFrame a formato Markdown."""
    md_table = dataframe.to_markdown(index=False)
    display(Markdown(md_table))

# Llamar a la función para mostrar la tabla
show_markdown_table(table.head(10))


| Title                            | Plot                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
|:---------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Kansas Saloon Smashers           | A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| Love by the Light of the Moon    | The moon, painted with a smiling face hangs over a park at night. A young couple walking past a fence learn on a railing and look up. The moon smiles. They embrace, and the moon's smile gets bigger. They then sit down on a bench by a tree. The moon's view is blocked, causing him to frown. In the last scene, the man fans the woman with his hat because the moon has left the sky and is perched over her shoulder to see everything better.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| The Martyred Presidents          | The film, just over a minute long, is composed of two shots. In the first, a girl sits at the base of an altar or tomb, her face hidden from the camera. At the center of the altar, a viewing portal displays the portraits of three U.S. Presidents—Abraham Lincoln, James A. Garfield, and William McKinley—each victims of assassination.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
|                                  | In the second shot, which runs just over eight seconds long, an assassin kneels feet of Lady Justice.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| Terrible Teddy, the Grizzly King | Lasting just 61 seconds and consisting of two shots, the first shot is set in a wood during winter. The actor representing then vice-president Theodore Roosevelt enthusiastically hurries down a hillside towards a tree in the foreground. He falls once, but rights himself and cocks his rifle. Two other men, bearing signs reading "His Photographer" and "His Press Agent" respectively, follow him into the shot; the photographer sets up his camera. "Teddy" aims his rifle upward at the tree and fells what appears to be a common house cat, which he then proceeds to stab. "Teddy" holds his prize aloft, and the press agent takes notes. The second shot is taken in a slightly different part of the wood, on a path. "Teddy" rides the path on his horse towards the camera and out to the left of the shot, followed closely by the press agent and photographer, still dutifully holding their signs.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| Jack and the Beanstalk           | The earliest known adaptation of the classic fairytale, this films shows Jack trading his cow for the beans, his mother forcing him to drop them in the front yard, and beig forced upstairs. As he sleeps, Jack is visited by a fairy who shows him glimpses of what will await him when he ascends the bean stalk. In this version, Jack is the son of a deposed king. When Jack wakes up, he finds the beanstalk has grown and he climbs to the top where he enters the giant's home. The giant finds Jack, who narrowly escapes. The giant chases Jack down the bean stalk, but Jack is able to cut it down before the giant can get to safety. He falls and is killed as Jack celebrates. The fairy then reveals that Jack may return home as a prince.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| Alice in Wonderland              | Alice follows a large white rabbit down a "Rabbit-hole". She finds a tiny door. When she finds a bottle labeled "Drink me", she does, and shrinks, but not enough to pass through the door. She then eats something labeled "Eat me" and grows larger. She finds a fan when enables her to shrink enough to get into the "Garden" and try to get a "Dog" to play with her. She enters the "White Rabbit's tiny House," but suddenly resumes her normal size. In order to get out, she has to use the "magic fan."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|                                  | She enters a kitchen, in which there is a cook and a woman holding a baby. She persuades the woman to give her the child and takes the infant outside after the cook starts throwing things around. The baby then turns into a pig and squirms out of her grip. "The Duchess's Cheshire Cat" appears and disappears a couple of times to Alice and directs her to the Mad Hatter's "Mad Tea-Party." After a while, she leaves.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
|                                  | The Queen invites Alice to join the "ROYAL PROCESSION": a parade of marching playing cards and others headed by the White Rabbit. When Alice "unintentionally offends the Queen", the latter summons the "Executioner". Alice "boxes the ears", then flees when all the playing cards come for her. Then she wakes up and realizes it was all a dream.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| The Great Train Robbery          | The film opens with two bandits breaking into a railroad telegraph office, where they force the operator at gunpoint to have a train stopped and to transmit orders for the engineer to fill the locomotive's tender at the station's water tank. They then knock the operator out and tie him up. As the train stops it is boarded by the bandits‍—‌now four. Two bandits enter an express car, kill a messenger and open a box of valuables with dynamite; the others kill the fireman and force the engineer to halt the train and disconnect the locomotive. The bandits then force the passengers off the train and rifle them for their belongings. One passenger tries to escape but is instantly shot down. Carrying their loot, the bandits escape in the locomotive, later stopping in a valley where their horses had been left.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|                                  | Meanwhile, back in the telegraph office, the bound operator awakens, but he collapses again. His daughter arrives bringing him his meal and cuts him free, and restores him to consciousness by dousing him with water.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
|                                  | There is some comic relief at a dance hall, where an Eastern stranger is forced to dance while the locals fire at his feet. The door suddenly opens and the telegraph operator rushes in to tell them of the robbery. The men quickly form a posse, which overtakes the bandits, and in a final shootout kills them all and recovers the stolen mail.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| The Suburbanite                  | The film is about a family who move to the suburbs, hoping for a quiet life. Things start to go wrong, and the wife gets violent and starts throwing crockery, leading to her arrest.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| The Little Train Robbery         | The opening scene shows the interior of the robbers' den. The walls are decorated with the portraits of notorious criminals and pictures illustrating the exploits of famous bandits. Some of the gang are lounging about, while others are reading novels and illustrated papers. Although of youthful appearance, each is dressed like a typical Western desperado. The "Bandit Queen," leading a blindfolded new recruit, now enters the room. He is led to the center of the room, raises his right hand and is solemnly sworn in. When the bandage is removed from his eyes he finds himself looking into the muzzles of a dozen or more 45's. The gang then congratulates the new member and heartily shake his hand. The "Bandit Queen" who is evidently the leader of the gang, now calls for volunteers to hold up a train. All respond, but she picks out seven for the job who immediately leave the cabin.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
|                                  | The next scene shows the gang breaking into a barn. They steal ponies and ride away. Upon reaching the place agreed upon they picket their ponies and leaving them in charge of a trusted member proceed to a wild mountain spot in a bend of the railroad, where the road runs over a steep embankment. The spot is an ideal one for holding up a train. Cross ties are now placed on the railroad track and the gang hide in some bushes close by and wait for the train. The train soon approaches and is brought to a stop. The engineer leaves his engine and proceeds to remove the obstruction on the track. While he is bending over one of the gang sneaks up behind them and hits him on the head with an axe, and knocks him senseless down the embankment, while the gang surround the train and hold up the passengers. After securing all the "valuables," consisting principally of candy and dolls, the robbers uncouple the engine and one car and make their escape just in time to avoid a posse of police who appear on the scene. Further up the road they abandon the engine and car, take to the woods and soon reach their ponies.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|                                  | In the meantime the police have learned the particulars of the hold-up from the frightened passengers and have started up the railroad tracks after the fleeing robbers. The robbers are next seen riding up the bed of a shallow stream and finally reach their den, where the remainder of the gang have been waiting for them. Believing they have successfully eluded their pursuers, they proceed to divide the "plunder." The police, however, have struck the right trail and are in close pursuit. While the "plunder" is being divided a sentry gives the alarm and the entire gang, abandoning everything, rush from the cabin barely in time to escape capture. The police make a hurried search and again start in pursuit. The robbers are so hard pressed that they are unable to reach their ponies, and are obliged to take chances on foot. The police now get in sight of the fleeing robbers and a lively chase follows through tall weeds, over a bridge and up a steep hill. Reaching a pond the police are close on their heels. The foremost robbers jump in clothes and all and strike out for the opposite bank. Two hesitate and are captured. Boats are secured and after an exciting tussle the entire gang is rounded up. In the mix up one of the police is dragged overboard. The final scene shows the entire gang of bedraggled and crestfallen robbers tied together with a rope and being led away by the police. Two of the police are loaded down with revolvers, knives and cartridge belts, and resemble walking aresenals. As a fitting climax a confederate steals out of the woods, cuts the rope and gallantly rescues the "Bandit Queen." |
| The Night Before Christmas       | Scenes are introduced using lines of the poem.[2] Santa Claus, played by Harry Eytinge, is shown feeding real reindeer[4] and finishes his work in the workshop. Meanwhile, the children of a city household hang their stockings and go to bed, but unable to sleep they engage in a pillow fight. Santa Claus leaves his home on a sleigh with his reindeer. He enters the children's house through the chimney, and leaves the presents. The children come down the stairs and enjoy their presents.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |

### **Instrucción**
#### **Parte 1: Construcción Manual de un Índice Invertido**
1. Cargar los datos en Python

In [8]:
import pandas as pd
import re
from collections import defaultdict

# Cargar el archivo CSV
file_path = r'C:\Users\USER\.cache\kagglehub\datasets\jrobischon\wikipedia-movie-plots\versions\1\wiki_movie_plots_deduped.csv'
df = pd.read_csv(file_path)

# Seleccionar solo las columnas necesarias
data = df[['Title', 'Plot']].head(36000)  # Limitar a 100 filas para pruebas

2. Construir un índice invertido manualmente:
   - Procesa las tramas de las películas para crear un índice invertido.
   - Normaliza el texto de las tramas.

In [9]:
# # Revisar si hay valores nulos
# print(f"Valores nulos antes de limpiar:\n{data.isnull().sum()}")

# # Eliminar filas con valores nulos en Title o Plot
# data = data.dropna(subset=['Title', 'Plot'])

# # Normalizar el texto (minúsculas, eliminar puntuación y caracteres especiales)
# def normalize_text(text):
#     text = text.lower()  # Convertir a minúsculas
#     text = re.sub(r'[^\w\s]', '', text)  # Eliminar puntuación
#     return text

# data['Normalized Plot'] = data['Plot'].apply(normalize_text)

# # Mostrar un ejemplo del texto normalizado
# data[['Title', 'Normalized Plot']].head()

In [10]:
import re

def normalize_text(text):
    """
    Normaliza el texto:
    - Convierte a minúsculas.
    - Elimina signos de puntuación.
    - Elimina números.
    - Elimina espacios adicionales.
    - Elimina palabras repetidas.
    """
    # Convertir a minúsculas
    text = text.lower()
    
    # Eliminar signos de puntuación y números
    text = re.sub(r'[^\w\s]', '', text)  # Puntuaciones
    text = re.sub(r'\d+', '', text)     # Números
    
    # Eliminar espacios adicionales
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Eliminar palabras repetidas manteniendo el orden
    palabras = text.split()
    palabras_unicas = []
    for palabra in palabras:
        if palabra not in palabras_unicas:
            palabras_unicas.append(palabra)
    
    # Reconstruir el texto limpio
    return ' '.join(palabras_unicas)

In [13]:
# def normalize_text(text):
#     """Normaliza el texto: convierte a minúsculas y elimina caracteres especiales."""
#     text = text.lower()
#     text = re.sub(r'[^\w\s]', '', text)  # Elimina puntuaciones
#     return text

# # Construir el índice invertido
# inverted_index = defaultdict(list)

# for index, row in data.iterrows():
#     title = row['Title']
#     plot = normalize_text(row['Plot'])
#     words = plot.split()
    
#     for word in words:
#         if title not in inverted_index[word]:  # Evitar duplicados
#             inverted_index[word].append(title)

3. Realizar consultas en el índice invertido:
   - Implementa funciones que permitan buscar palabras clave y devolver los títulos de las películas que contienen dichas palabras.

In [14]:
# def search_in_index(query, index):
#     """Busca palabras clave en el índice invertido."""
#     query = normalize_text(query)
#     words = query.split()
#     results = set(index[words[0]]) if words[0] in index else set()

#     for word in words[1:]:
#         if word in index:
#             results = results.intersection(index[word])
#         else:
#             results = set()  # Si una palabra no existe, no hay resultados
#             break
    
#     return results

# # Ejemplo de búsqueda
# query = "saloon"
# result = search_in_index(query, inverted_index)

# print("Películas encontradas:", result)

In [15]:
from collections import defaultdict
import re
import pandas as pd

# Función para normalizar texto
def normalize_text(text):
    text = text.lower()  # Minúsculas
    text = re.sub(r'[^\w\s]', '', text)  # Eliminar puntuaciones
    text = re.sub(r'\d+', '', text)     # Eliminar números
    text = re.sub(r'\s+', ' ', text).strip()  # Espacios adicionales
    
    # Eliminar palabras repetidas
    palabras = text.split()
    palabras_unicas = []
    for palabra in palabras:
        if palabra not in palabras_unicas:
            palabras_unicas.append(palabra)
    return ' '.join(palabras_unicas)

# Cargar datos
file_path = r'C:\Users\USER\.cache\kagglehub\datasets\jrobischon\wikipedia-movie-plots\versions\1\wiki_movie_plots_deduped.csv'
df = pd.read_csv(file_path)
data = df[['Title', 'Plot']].head(36000)  # Seleccionar las columnas necesarias

# Normalizar tramas
data['Normalized Plot'] = data['Plot'].apply(normalize_text)

# Construir índice invertido manualmente
def construir_indice_invertido(data):
    """
    Construye un índice invertido a partir del dataframe procesado.
    """
    inverted_index = defaultdict(set)  # Usar set para eficiencia
    for _, row in data.iterrows():
        title = row['Title']
        plot = row['Normalized Plot']
        for word in plot.split():
            if len(word) >= 3:  # Evitar palabras cortas
                inverted_index[word].add(title)  # Añadir título
    
    # Convertir los conjuntos a listas
    return {key: list(value) for key, value in inverted_index.items()}

# Crear el índice invertido
indice_invertido = construir_indice_invertido(data)

# Guardar el índice invertido en un archivo
import json
with open('inverted_index.json', 'w', encoding='utf-8') as f:
    json.dump(indice_invertido, f, ensure_ascii=False, indent=4)

print("Índice invertido creado y guardado en 'inverted_index.json'")

Índice invertido creado y guardado en 'inverted_index.json'


In [16]:
def buscar_palabras(indice_invertido, consulta):
    """
    Busca términos en el índice invertido y devuelve resultados relevantes.
    """
    consulta = normalize_text(consulta)
    palabras = consulta.split()
    resultados = set(indice_invertido.get(palabras[0], []))
    
    for palabra in palabras[1:]:
        resultados.intersection_update(indice_invertido.get(palabra, []))
    return resultados

# Ejemplo de consulta
consulta = "saloon"
with open('inverted_index.json', 'r', encoding='utf-8') as f:
    indice_invertido = json.load(f)

resultados = buscar_palabras(indice_invertido, consulta)
print("Películas encontradas:", resultados)

Películas encontradas: {'The Quick and the Dead', 'Born to the Saddle', 'The Mighty McGurk', 'Way Out West', 'The Proud Ones', 'Tom Horn', 'Death on the Nile', 'Ruggles of Red Gap', 'High Plains Drifter', 'Johnny Guitar', 'Melody Ranch', 'The Assassination of Jesse James by the Coward Robert Ford', 'The Ballad of Little Jo', 'Montana Belle', 'The Maverick Queen', 'No Name on the Bullet', 'Boss Engira Bhaskaran', 'Warlock', 'The Roaring Twenties', 'Belle of the Yukon', 'The Younger Brothers', "It's a Dog's Life", 'Raw Edge', 'Support Your Local Sheriff', 'Dick Tracy Meets Gruesome', 'Moran of the Lady Letty', 'The Ballad of Cable Hogue', 'The Woman of the Town', 'Frontier Outlaws', 'The Spoilers', 'Pierre of the Plains', 'The Terror of Tiny Town', 'The Life and Times of Judge Roy Bean', 'The Frozen North', 'Posse from Hell', 'The Decks Ran Red', 'The Lawless Breed', 'Shane', 'Quantez', 'A Gunfight', 'Oh! Susanna', 'The Ox-Bow Incident', 'Johnny Reno', 'The Arizona Kid', 'The Carson City

#### **Parte 2: Usar Whoosh para Indexación y Recuperación**
1. Configurar un índice con Whoosh:
   - Define un esquema con los campos Title y Plot.
   - Crea un índice y agrega las películas seleccionadas.
2. Agregar documentos al índice:
   - Usa las funciones de escritura de Whoosh para indexar los títulos y tramas.
3. Realizar consultas:
   - Busca películas relevantes usando palabras clave relacionadas con las tramas (e.g., “dinosaurs”, “cyborg”).

#### **Parte 3: Usar Elasticsearch para Indexación y Recuperación**
1. Iniciar Elasticsearch con Docker
2. Configurar y definir Elasticsearch en Python:
   - Conecta Elasticsearch y crea un índice con los campos Title y Plot.
3. Indexar documentos:
   - Inserta cada película en el índice de Elasticsearch, usando el esquema definido.
4. Realizar consultas:
   - Realiza búsquedas utilizando tramas relacionadas con palabras clave como “time travel” o “genetic engineering”.